In [1]:
from utils import TestResults,TrainSplit
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, log_loss

import numpy as np
import optuna

In [2]:
X_train, X_test, y_train, y_test = TrainSplit().get_split()

Loading the data
Categorial Feature Imputed


NUM IMPUTE: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 52.81it/s]


Spliting the Data based on time
X_train:(3387880, 80), X_test:(97972, 80) , y_train:(3387880, 2) , y_test:(97972, 2)


In [3]:
cat_feat = TrainSplit.CATEGORIES + TrainSplit.BINARY
num_feat = TrainSplit.NUMERICAL
all_feat = cat_feat + num_feat

In [4]:
params_all_feat = {'max_depth': 7, 'learning_rate': 0.05198517520669243, 'n_estimators': 606,
                   'tree_method':'gpu_hist','objective':'binary:logistic'}
params_cat_feat = {'max_depth': 4, 'learning_rate': 0.2972844912008205, 'n_estimators': 257,
                   'tree_method':'gpu_hist','objective':'binary:logistic'}
params_num_feat = {'max_depth': 5, 'learning_rate': 0.18539658586808733, 'n_estimators': 603,
                   'tree_method':'gpu_hist','objective':'binary:logistic'}

In [5]:
model_all_feat = XGBClassifier(**params_all_feat)
model_all_feat.fit(X_train[all_feat],y_train[TrainSplit.IS_INSTALLED[0]])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05198517520669243,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=606, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [6]:
import pandas as pd
import numpy as np

test = pd.read_csv('../Data/test/000000000000.csv',sep='\t')
test['f_30'].fillna(test['f_30'].mode()[0],inplace=True)
test['f_31'].fillna(test['f_31'].mode()[0],inplace=True)
fmiss = "f_43,f_51,f_58,f_59,f_64,f_65,f_66,f_67,f_68,f_69,f_70".split(',')
for f in fmiss:
    test[f].fillna(test[f].mean(),inplace=True)

In [7]:
install_pred_all = model_all_feat.predict_proba(test[all_feat])[:,1]

In [8]:
model_cat_feat = XGBClassifier(**params_cat_feat)
model_cat_feat.fit(X_train[cat_feat],y_train[TrainSplit.IS_INSTALLED[0]])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2972844912008205,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=257, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [9]:
install_pred_cat = model_cat_feat.predict_proba(test[cat_feat])[:,1]

In [10]:
install_pred_cat

array([0.30913222, 0.31705943, 0.02517575, ..., 0.17822418, 0.05227256,
       0.12873745], dtype=float32)

In [11]:
install_pred_all

array([0.19879813, 0.28428742, 0.0775891 , ..., 0.015904  , 0.12681367,
       0.05524745], dtype=float32)

In [12]:
model_num_feat = XGBClassifier(**params_num_feat)
model_num_feat.fit(X_train[num_feat],y_train[TrainSplit.IS_INSTALLED[0]])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.18539658586808733,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=603, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [13]:
install_pred_num = model_num_feat.predict_proba(test[num_feat])[:,1]

In [14]:
install_pred_num

array([0.08649381, 0.27345726, 0.2478346 , ..., 0.2819063 , 0.3294385 ,
       0.01551908], dtype=float32)

In [15]:
stack_test = np.vstack([install_pred_all,install_pred_cat,install_pred_num]).T

In [16]:
stack_test.shape

(160973, 3)

In [17]:
y_pred_all_pred = model_all_feat.predict_proba(X_train[all_feat])[:,1]
y_pred_cat_pred = model_cat_feat.predict_proba(X_train[cat_feat])[:,1]
y_pred_num_pred = model_num_feat.predict_proba(X_train[num_feat])[:,1]

In [18]:
y_pred_all_pred

array([0.0689008 , 0.24267854, 0.27915812, ..., 0.09421918, 0.3630598 ,
       0.2659234 ], dtype=float32)

In [19]:
y_pred_cat_pred

array([0.03071583, 0.2652175 , 0.24187617, ..., 0.1443992 , 0.2510125 ,
       0.27600548], dtype=float32)

In [20]:
y_pred_num_pred

array([0.07044848, 0.20469281, 0.19933815, ..., 0.03526987, 0.3520708 ,
       0.2862676 ], dtype=float32)

In [21]:
y_train[TrainSplit.IS_INSTALLED[0]]

0          1
1          0
2          1
3          0
4          0
          ..
3485847    0
3485848    0
3485849    0
3485850    0
3485851    1
Name: is_installed, Length: 3387880, dtype: int64

In [22]:
y_pred_valid_all_pred = model_all_feat.predict_proba(X_test[all_feat])[:,1]
y_pred_valid_cat_pred = model_cat_feat.predict_proba(X_test[cat_feat])[:,1]
y_pred_valid_num_pred = model_num_feat.predict_proba(X_test[num_feat])[:,1]

# Dates can also be added here 

In [37]:
X_stack_train = np.vstack([X_train['f_1'].to_numpy(),y_pred_all_pred,y_pred_cat_pred,y_pred_num_pred]).T
X_stack_valid = np.vstack([X_test['f_1'].to_numpy(),y_pred_valid_all_pred,y_pred_valid_cat_pred,y_pred_valid_num_pred]).T

In [38]:
X_stack_train.shape

(3387880, 4)

In [39]:
X_stack_valid.shape

(97972, 4)

In [ ]:
from sklearn.linear_model import LogisticRegression

def objective(trail):
    params = {  'C': trail.suggest_float('C',0.2,2),
                'class_weight':'balanced',# Try None as well
                'n_jobs': -1,
                'max_iter':2000
                # 'penalty': trail.suggest_categorical('penalty', ["l1", "l2", 'elasticnet']),
             }
    
    target = TrainSplit.IS_INSTALLED[0]
    model = LogisticRegression(**params)
    model.fit(X_stack_train,y_train[TrainSplit.IS_INSTALLED[0]])
    y_pred = model.predict(X_stack_valid)
    score = log_loss(y_test[target],y_pred)
    return score

In [40]:
def objective(trail):
    params = {
        'max_depth':trail.suggest_int('max_depth',3,8),
        'learning_rate':trail.suggest_float('learning_rate',0.05,0.3),
        'n_estimators':trail.suggest_int('n_estimators',10,650),
        'tree_method':'gpu_hist',
        'objective':'binary:logistic'
    }
    # print(use_features)
    target = TrainSplit.IS_INSTALLED[0]
    model = XGBClassifier(**params)
    model.fit(X_stack_train,y_train[target])
    y_pred = model.predict(X_stack_valid)
    score = log_loss(y_test[target],y_pred)
    # print(model.feature_names_in_)
    return score

In [41]:
import optuna
study_stack = optuna.create_study(direction='minimize',study_name='stack_log_loss')
study_stack.optimize(objective, n_trials=30)

[I 2023-05-25 08:51:13,500] A new study created in memory with name: stack_log_loss
[I 2023-05-25 08:51:17,773] Trial 0 finished with value: 6.049929463787792 and parameters: {'max_depth': 6, 'learning_rate': 0.20974285467002218, 'n_estimators': 390}. Best is trial 0 with value: 6.049929463787792.
[I 2023-05-25 08:51:19,961] Trial 1 finished with value: 6.061210246713174 and parameters: {'max_depth': 6, 'learning_rate': 0.2230584389267708, 'n_estimators': 163}. Best is trial 0 with value: 6.049929463787792.
[I 2023-05-25 08:51:23,818] Trial 2 finished with value: 6.034770722487274 and parameters: {'max_depth': 6, 'learning_rate': 0.1394877040182615, 'n_estimators': 357}. Best is trial 2 with value: 6.034770722487274.
[I 2023-05-25 08:51:26,524] Trial 3 finished with value: 6.053103196520324 and parameters: {'max_depth': 4, 'learning_rate': 0.12077593478049888, 'n_estimators': 325}. Best is trial 2 with value: 6.034770722487274.
[I 2023-05-25 08:51:27,575] Trial 4 finished with value: 6

In [32]:
params = {'max_depth': 4, 'learning_rate': 0.10151446144603166, 'n_estimators': 23,
          'tree_method':'gpu_hist','objective':'binary:logistic'}
model = XGBClassifier(**params)
model.fit(X_stack_train,y_train[TrainSplit.IS_INSTALLED[0]])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.10151446144603166,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=23, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [33]:
install_pred = model.predict_proba(stack_test)[:,1]

In [34]:
install_pred.shape

(160973,)

In [35]:
TestResults(row_id=test['f_0'],is_click=np.random.random(install_pred.shape[0]),
            is_install=install_pred,model_name="xgb_stack_all_cat_num_xgb",config=params)

Saved the test result to csv file as xgb_stack_all_cat_num_xgb_2023-05-25 06:19:35.109117.csv
Saved the model config to json file as xgb_stack_all_cat_num_xgb_2023-05-25 06:19:35.109117.json
